In [354]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [306]:
dom1 = []
dom2 = []
test_set = []
with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('../data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("../data/sample.csv")

with open('../data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [287]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [243]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2150


## Resampling

In [406]:
#让dom2的label，model分布都平均，差不多307一组，st 307行是label 1 标为model 5，再从dom1里取307个label1和307个label0，人工的model是nan

dom2_0 = dom2.loc[dom2['label']==0]
dom2_1 = dom2.loc[dom2['label']==1]

length_dom2_1 = len(dom2_1)
mol_num = len(dom2['model'].value_counts())
sam_dom2_0 = dom2_0.groupby('model').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sam_dom2_0.reset_index(drop=True, inplace=True)
dom2_new = result = pd.concat([dom2_1, sam_dom2_0])

sampled_dom1 = dom1.groupby('label').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sampled_dom1.reset_index(drop=True, inplace=True)

df = result = pd.concat([sampled_dom1, dom2_new])

In [407]:
print(df['label'].value_counts())
print(df['model'].value_counts())


label
0    2456
1    2456
Name: count, dtype: int64
model
0.0    307
1.0    307
2.0    307
3.0    307
4.0    307
5.0    307
6.0    307
Name: count, dtype: int64


In [410]:
df = df.sample(frac=1).reset_index(drop=True)

In [411]:
df = df.drop(['model'], axis=1)

In [412]:
def to_sparse_matrix(sequences, vocab_size):
    indptr = [0]
    indices = []
    data = []
    for seq in sequences:
        feature_counter = {}
        for index in seq:
            if index != 0:  # Skip 0s, other words
                if index not in feature_counter:
                    feature_counter[index] = 1
                else:
                    feature_counter[index] += 1
        indices.extend(feature_counter.keys())
        data.extend(feature_counter.values())
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int, shape=(len(sequences), vocab_size + 1))

vocab_size = 4999
X = to_sparse_matrix(df['text'], vocab_size)

In [413]:
y = df['label']

selector = SelectKBest(chi2, k=1000)
X_new = selector.fit_transform(X, y)

In [414]:
X_pre = to_sparse_matrix(test_set['text'], vocab_size)
X_pre_new = selector.transform(X_pre)

In [301]:
# # 试一下转成mean，std为feature
# df['feature_mean'] = df['text'].apply(np.mean)
# df['feature_std'] = df['text'].apply(np.std)
# X = df[['feature_mean', 'feature_std']]
# y = df['label']

# test_set['feature_mean'] = test_set['text'].apply(np.mean)
# test_set['feature_std'] = test_set['text'].apply(np.std)
# X_pre = test_set[['feature_mean', 'feature_std']]

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)
model = svm.SVC()
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Test score: {score}")

Test score: 0.6907426246185148


In [424]:
test_pre = model.predict(X_pre_new)

In [438]:
result = pd.DataFrame({
    'class': test_pre,
})
result['id'] = result.index
result = result[['id'] + [col for col in result.columns if col != 'id']]


In [440]:
result

,id,class
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
995,995,1
996,996,1
997,997,1
998,998,0


In [378]:
sam

,id,class
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
995,995,1
996,996,0
997,997,0
998,998,0


In [382]:
result.to_csv('result.csv', index=False)


In [230]:
# selector = SelectKBest(f_classif, k=200)  
# X_new = selector.fit_transform(X, y)

# mask = selector.get_support()
# selected_features = X.columns[mask]
# print('Selected features:', selected_features.tolist())

Selected features: ['feature_mean', 'feature_std']


In [343]:
# from sklearn.linear_model import LogisticRegression
# import numpy as np
# from sklearn.model_selection import train_test_split
# features = df['text']
# labels = df['label']
# max_length = max([len(f) for f in features])

# features_padded = np.array([f + [0]*(max_length - len(f)) for f in features])
# X_train, X_test, y_train, y_test = train_test_split(features_padded, labels, test_size=0.2, random_state=42)
# model = LogisticRegression()
# model.fit(X_train, y_train)

# score = model.score(X_test, y_test)
# print(f"Test score: {score}")


# x_pre = test_set['text']
# max_length = max([len(f) for f in x_pre])

# features_padded = np.array([f + [0]*(max_length - len(f)) for f in x_pre])

# len(features_padded[0])